In [1]:
from __future__ import print_function
from collections import deque
from tensorflow.python.framework import ops
from rl_reinforce import REINFORCEothello
from othello_net import *
from othello_rules import *
from feature_extractor import *
import os, random, sys, gym
import tensorflow as tf
import numpy as np
from datetime import datetime
from example_states import *
from training_utils import *
np.set_printoptions(precision=2)
ops.reset_default_graph()

In [2]:
def board_to_features(board, p, new_move = False, prev_filled = None):
    #TODO: Add a lot more heuristics!
    o                = p * (-1)
    p_grid             = np.zeros((8,8))
    o_grid           = np.zeros((8,8))
    empties                 = np.zeros((8,8))
    p_constant         = np.zeros((8,8))
    zeros                   = np.zeros((8,8))
    legal_move_grid         = np.zeros((8,8))
    ones                    = np.ones((8,8))
    corners                 = corner_board()
    x_squares               = np.zeros((8,8))
    c_grid                  = c_squares()
    mobility                = np.zeros((8,8))
    edges_a                 = edges_board_a()
    edges_b                 = edges_board_b()
    frontier                = np.zeros((8,8))
    sum_p_stability    = np.zeros((8,8))
    sum_o_stability  = np.zeros((8,8))
    current_stability       = np.zeros((8,8))
    coin_parity             = np.zeros((8,8))
    main_diagonal_1         = np.identity(8)
    main_diagonal_2         = np.fliplr(np.identity(8))
    
    corner_2x4_1    = corner_2x4()
    corner_2x4_2    = np.rot90(corner_2x4_1)
    corner_2x4_3    = np.rot90(corner_2x4_2)
    corner_2x4_4    = np.rot90(corner_2x4_3)
    corner_2x4_5    = np.transpose(corner_2x4_1)
    corner_2x4_6    = np.transpose(corner_2x4_2)
    corner_2x4_7    = np.transpose(corner_2x4_3)
    corner_2x4_8    = np.transpose(corner_2x4_4)
    x_squares[1][1] = 1
    x_squares[6][1] = 1
    x_squares[6][6] = 1
    x_squares[1][6] = 1
    
    #stability = np.zeros((8,8))
    for i in range(8):
        for j in range(8):
            #move_string = str((i+1)*10+j)
            #mobility[i][j] = o_mobility_after_move(board, move_string, p)
            if p == -1:
                p_constant[i][j] = 1
            if board[i][j] == p:
                p_grid[i][j]     = 1
            elif board[i][j] != 0:
                o_grid[i][j]   = 1
            else:
                empties[i][j]         = 1
    
    #TODO: Speed up stability calculations such that it utilizes it being monotonically increasing
    if a_corner_is_filled(board):
        ind_rows          = filled_rows(board, new_move)    
        ind_columns       = filled_columns(board, new_move)
        ind_NW            = filled_NW(board)
        ind_NE            = filled_NE(board, new_move)
        if prev_filled is not None:
            initial_rows      = get_filled_row_features(ind_rows, new_move, prev_filled[:,:,0])
            initial_columns   = get_filled_column_features(ind_columns, new_move, prev_filled[:,:,1])
            initial_NE        = get_filled_NE_features(ind_NE, new_move, prev_filled[:,:,3])
        else:
            initial_rows      = get_filled_row_features(ind_rows)
            initial_columns   = get_filled_column_features(ind_columns)
            initial_NE        = get_filled_NE_features(ind_NE)
        initial_NW        = get_filled_NW_features(ind_NW)
        
        current_stability = get_stability_features(board, p, initial_rows, initial_columns, initial_NW, initial_NE)
    for move in find_legal_moves(board, p):
        row, col = str_move_to_ints(move)
        legal_move_grid[row,col]  = 1
        mobility[row][col]        = opponent_mobility_after_move(board, move, p)
        board_after_p_move   = make_move(board, move, p)
        coin_parity[row, col]     = np.sum(board_after_p_move == p) - np.sum(board_after_p_move == p*(-1))
        board_after_o_move = make_move(board, move, o)
        frontier[row, col]        = calculate_frontier(board, (row, col))
        # Only calculate stability if any of the twelve indicated corner cells are occupied
        if a_corner_is_filled(board):
            ind_rows     = filled_rows(board_after_p_move, move)
            ind_cols     = filled_columns(board_after_p_move, move)
            ind_NW       = filled_NW(board_after_p_move)
            ind_NE       = filled_NE(board_after_p_move, move)
            fill_rows    = get_filled_row_features(ind_rows, move, initial_rows)
            fill_cols    = get_filled_column_features(ind_cols, move, initial_columns)
            fill_NW      = get_filled_NW_features(ind_NW)
            fill_NE      = get_filled_NE_features(ind_NE, move, initial_NE)
            potential_p_sta = get_stability_features(board_after_p_move, p, fill_rows, fill_cols, fill_NW, fill_NE)
            potential_o_sta = get_stability_features(board_after_o_move, o, fill_rows, fill_cols, fill_NW, fill_NE)
            sum_p_stability[row][col] = np.sum(potential_p_sta)
            sum_o_stability[row][col] = np.sum(potential_o_sta)
            
    return np.dstack((p_grid, o_grid, empties,
                           p_constant, zeros, legal_move_grid,
                          corners, x_squares, c_grid,
                          ones, mobility, edges_a, edges_b,
                          sum_p_stability, sum_o_stability,
                          current_stability, frontier, coin_parity,
                          main_diagonal_1, main_diagonal_2, corner_2x4_1,
                          corner_2x4_2, corner_2x4_3, corner_2x4_4,
                          corner_2x4_5, corner_2x4_6, corner_2x4_7,
                          corner_2x4_8)), np.dstack((initial_rows, initial_columns, initial_NW, initial_NE))


initial = ffo_endgame_45()
initial[0, 1] = 1
initial[1, 0] = 1
#print(initial)
#print(find_legal_moves(initial, 1))
#print("old filled columns")
#print(filled[:,:,1])
a = datetime.now()
oldf_initial = board_to_input(initial, -1)
b = datetime.now()
new_move = '73'
newf_initial, filled = board_to_features(initial, -1)

test = make_move(initial, new_move, 1)
c = datetime.now()
print("second call")
fast_initial, filled = board_to_features(test, -1, new_move, filled)
d = datetime.now()
print(initial)
print("old method time: " + str(b-a))
print("new method time: " + str(d-c))
#TODO: ég er að láta alla SCANNA of langt, stundum þurfa þeir bara að finna eitt 0 og þá eru þeir búnir
print(np.array_equal(oldf_initial, fast_initial))
print(test)
pressibus= gen_pressibus_example()
init_pressibus, init_filled = board_to_features(pressibus, 1)
#print(find_legal_moves(pressibus, 1))
pres_move = '47'
pressibus = make_move(pressibus, '47', 1)
fast_pressibus, filledz = board_to_features(pressibus, 1, '47', init_filled)


second call
[[ 0.  1. -1. -1. -1. -1. -1.  0.]
 [ 1.  0. -1. -1. -1. -1.  0.  0.]
 [ 0.  1.  1.  1. -1. -1.  0.  0.]
 [ 0.  1.  1. -1. -1. -1. -1.  0.]
 [ 0.  1.  1. -1. -1. -1.  1.  0.]
 [ 1.  1.  1.  1. -1.  1.  1.  0.]
 [ 0.  0.  0. -1.  1. -1.  0.  0.]
 [ 0.  0. -1. -1. -1. -1. -1.  0.]]
old method time: 0:00:00.013911
new method time: 0:00:00.010792
False
[[ 0.  1. -1. -1. -1. -1. -1.  0.]
 [ 1.  0. -1. -1. -1. -1.  0.  0.]
 [ 0.  1.  1.  1. -1. -1.  0.  0.]
 [ 0.  1.  1. -1. -1. -1. -1.  0.]
 [ 0.  1.  1. -1. -1. -1.  1.  0.]
 [ 1.  1.  1.  1. -1.  1.  1.  0.]
 [ 0.  0.  1.  1.  1. -1.  0.  0.]
 [ 0.  0. -1. -1. -1. -1. -1.  0.]]


In [3]:
from feature_extractor_old import *
e = datetime.now()
oldie_features = board_to_input_old(test, -1)
f = datetime.now()
print("real old time: " + str(f-e))
old_pressibus = board_to_input(pressibus, 1)

print(np.array_equal(oldie_features, fast_initial))
print(np.array_equal(old_pressibus, fast_pressibus))
print(np.array_equal(oldie_features, fast_pressibus))

real old time: 0:00:00.014942
True
False
False


In [4]:
print(pressibus)
print(test)

[[ 1.  1. -1.  1.  0.  0.  0.  0.]
 [ 1. -1.  1.  0.  0.  0.  0.  1.]
 [ 1.  1.  1.  0.  0.  0.  1.  0.]
 [-1.  1.  1.  0.  1.  1.  1.  1.]
 [ 1. -1. -1.  1. -1.  0.  1.  1.]
 [ 0. -1. -1. -1.  0.  0.  1.  1.]
 [-1. -1.  1.  1. -1.  1.  1.  1.]
 [ 1.  1. -1.  1.  0.  0.  1.  1.]]
[[ 0.  1. -1. -1. -1. -1. -1.  0.]
 [ 1.  0. -1. -1. -1. -1.  0.  0.]
 [ 0.  1.  1.  1. -1. -1.  0.  0.]
 [ 0.  1.  1. -1. -1. -1. -1.  0.]
 [ 0.  1.  1. -1. -1. -1.  1.  0.]
 [ 1.  1.  1.  1. -1.  1.  1.  0.]
 [ 0.  0.  1.  1.  1. -1.  0.  0.]
 [ 0.  0. -1. -1. -1. -1. -1.  0.]]
